In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Nov 21 14:49:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown --id 1Z_RSO1UKRfJXdL47MthB0iPpSXRNqd1D --output "data.zip"
!unzip -q "data.zip"

Downloading...
From: https://drive.google.com/uc?id=1Z_RSO1UKRfJXdL47MthB0iPpSXRNqd1D
To: /content/data.zip
100% 642M/642M [00:03<00:00, 205MB/s]


In [ ]:
! git clone https://github.com/eriklindernoren/PyTorch-GAN
# cd PyTorch-GAN/
# ! sudo pip3 install -r requirements.txt/


Cloning into 'PyTorch-GAN'...
remote: Enumerating objects: 1283, done.
remote: Total 1283 (delta 0), reused 0 (delta 0), pack-reused 1283
Receiving objects: 100% (1283/1283), 68.04 MiB | 9.82 MiB/s, done.
Resolving deltas: 100% (751/751), done.


In [ ]:
# ! git clone https://github.com/eriklindernoren/PyTorch-GAN
%cd PyTorch-GAN/
! sudo pip3 install -r requirements.txt


[Errno 2] No such file or directory: 'PyTorch-GAN/'
/content/PyTorch-GAN


In [ ]:
%cd PyTorch-GAN/
! sudo pip3 install -r requirements

/content/PyTorch-GAN
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements'


In [ ]:
%cd implementations/acgan/
# $ python3 acgan.py÷


SyntaxError: ignored

In [ ]:
! cp -r /content/drive/MyDrive/DLCV/HW2/models_3 /content/

In [ ]:
from torchvision import datasets

import os, glob
import numpy as np
import pandas as pd
import torch

# from glob import glob
from PIL import Image

train_data = sorted(glob.glob(os.path.join('/content/hw2_data/digits/mnistm/train', '*.png')))
test_data = sorted(glob.glob(os.path.join('/content/hw2_data/digits/mnistm/test', '*.png')))

train_X = []

for fn in train_data:
  image = Image.open(fn)
  train_X.append(image)

for fn in test_data:
  image = Image.open(fn)
  train_X.append(image)

# for fn in train_data:
#   image = Image.open(fn)
#   train_X.append(np.array(image))

# for fn in test_data:
#   image = Image.open(fn)
#   train_X.append(np.array(image))

# load data
# train_X = np.load("all_img_flip.npy")
train_attr = pd.read_csv("/content/hw2_data/digits/mnistm/train.csv")
test_attr = pd.read_csv("/content/hw2_data/digits/mnistm/test.csv")

print(type(train_attr.iloc[2,1]))

smiling_attr = []

for i in range(60000):
  smiling_attr.append(train_attr.iloc[i,1])

for j in range(10000):
  smiling_attr.append(test_attr.iloc[j,1])




print(len(smiling_attr), type(smiling_attr))
print(type(smiling_attr))
# print(smiling_attr.shape)
print(smiling_attr[0:6])

# smiling_attr = np.hstack((np.repeat(np.array(train_attr["Smiling"]),2) 
#                           ,np.repeat(np.array(test_attr["Smiling"]),2)
#                          ))
# smiling_attr = np.hstack((np.array(train_attr["Smiling"])
#                           ,np.array(test_attr["Smiling"]))
#                          )

# type transform
# img_X = torch.from_numpy(train_X).type(torch.FloatTensor)
# class_X = torch.from_numpy(smiling_attr).type(torch.FloatTensor).view(-1,1,1,1)

<class 'numpy.int64'>
70000 <class 'list'>
<class 'list'>
[5, 0, 4, 1, 9, 2]


In [ ]:
print(len(train_X))

70000


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class MyDataset(Dataset):
  def __init__(self, dir, csv):
    self.imgs = dir
    self.labels = csv
    self.transform = transforms.Compose([
      transforms.Resize(opt.img_size),
      transforms.ToTensor(),
      transforms.Normalize([0.5], [0.5])                                         
    ])

  def __getitem__(self, index):
    return self.transform(self.imgs[index]), self.labels[index]
  
  def __len__(self):
    return len(self.imgs)

In [ ]:
import torch
import numpy as np
import random

def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from argparse import Namespace

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True)
same_seeds(1126)

# parser = argparse.ArgumentParser()
# parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
# parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
# parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
# parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
# parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
# parser.add_argument("--n_classes", type=int, default=10, help="number of classes for dataset")
# parser.add_argument("--img_size", type=int, default=32, help="size of each image dimension")
# parser.add_argument("--channels", type=int, default=3, help="number of image channels")
# parser.add_argument("--sample_interval", type=int, default=400, help="interval between image sampling")
# opt = parser.parse_args()
# print(opt)

parameters = {
    "n_epochs": 200,
    "batch_size": 64,
    "lr": 0.0002,
    "b1": 0.5,
    "b2": 0.999,
    "n_cpu": 8,
    "latent_dim": 100,
    "n_classes": 10,
    "img_size": 64,
    "channels": 3,
    "sample_interval": 400,
}
opt = Namespace(**parameters)

cuda = True if torch.cuda.is_available() else False


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.latent_dim)

        self.init_size = opt.img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4

        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, opt.n_classes), nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label


# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()


if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
# os.makedirs("../../data/mnist", exist_ok=True)
# dataloader = torch.utils.data.DataLoader(
#     datasets.MNIST(
#         "../../data/mnist",
#         train=True,
#         download=True,
#         transform=transforms.Compose(
#             [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
#         ),
#     ),
#     batch_size=opt.batch_size,
#     shuffle=True,
# )
myset = MyDataset(train_X, smiling_attr)
dataloader = DataLoader(myset)


# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor


def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
        )
        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

RuntimeError: ignored

In [ ]:
torch.save(generator.state_dict(), "/content/models/ACG2_model_g.pkt")
torch.save(discriminator.state_dict(), "/content/models/ACG2_model_d.pkt")

# model = Generator().to('cuda')
# state = torch.load('/content/models_2/models_2/ACG2_model.pkt')
# model.load_state_dict(state)
# model.eval()

In [ ]:
torch.save(generator, "/content/models/ACG2_model_g_state.pkt")
torch.save(discriminator, "/content/models/ACG2_model_d_state.pkt")

In [ ]:
def load_checkpoint(checkpoint_path, model):
    state = torch.load(checkpoint_path, map_location = "cuda")
    # model.load_state_dict(state['state_dict'])
    model.load_state_dict(state)
    print('model loaded from %s' % checkpoint_path)

In [ ]:
load_checkpoint('/content/models/ACG2_model_g.pkt', generator)

model loaded from /content/models/ACG2_model_g.pkt


In [ ]:
# ! cp -r /content/images /content/drive/MyDrive/DLCV/HW2

In [ ]:
! cp -r /content/drive/MyDrive/DLCV/HW2/models_2 /content/models_2

In [ ]:
model = Generator().to('cuda')
state = torch.load('/content/models_2/models_2/ACG2_model.pkt')
model.load_state_dict(state)

In [ ]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F


def load_checkpoint(checkpoint_path, model):
    state = torch.load(checkpoint_path, map_location = "cuda")
    model.load_state_dict(state['state_dict'])
    print('model loaded from %s' % checkpoint_path)


class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


if __name__ == '__main__':
    
    # load digit classifier
    net = Classifier()
    path = "Classifier.pth"
    load_checkpoint(path, net)

    transform = transforms.Compose([
      transforms.Resize(28),
      transforms.ToTensor(),
      transforms.Normalize([0.5], [0.5])                                         
    ])

    # GPU enable
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print('Device used:', device)
    if torch.cuda.is_available():
        net = net.to(device)

    print(net)